In [105]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_predict
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score, confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

import time

In [106]:
#Read Data from soruces

# Read the sensor feature file (590 columns, no header)
df_features = pd.read_csv("C:/Users/Prajakta B/Desktop/SECOM_DATA/secom/secom.data",
                          delim_whitespace=True,  # or sep=' '
                          header=None)

# Read the labels file (two columns: label and timestamp)
df_labels = pd.read_csv("C:/Users/Prajakta B/Desktop/SECOM_DATA/secom/secom_labels.data",
                        delim_whitespace=True,
                        header=None)

# df_features.head()
# df_labels.head()

C:\Users\Prajakta B\AppData\Local\Temp\ipykernel_10720\885378504.py:4: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_features = pd.read_csv("C:/Users/Prajakta B/Desktop/SECOM_DATA/secom/secom.data",
C:\Users\Prajakta B\AppData\Local\Temp\ipykernel_10720\885378504.py:9: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_labels = pd.read_csv("C:/Users/Prajakta B/Desktop/SECOM_DATA/secom/secom_labels.data",


In [107]:
#Combining data and creating proper structure of table


# Loading dataset 
try:
    secom_data = pd.read_csv('C:/Users/Prajakta B/Desktop/SECOM_DATA/secom/secom.data', delim_whitespace=True, header=None)
    labels = pd.read_csv('C:/Users/Prajakta B/Desktop/SECOM_DATA/secom/secom_labels.data', delim_whitespace=True, header=None)
except FileNotFoundError as e:
    print(f"Error: {e}. Please verify file paths and names.")
    exit()
    
# Rename columns like feature_1,2.. target_1,2..
num_sensors = secom_data.shape[1]  # Number of sensor columns
new_columns = [f'feature_{i+1}' for i in range(num_sensors)] 
secom_data.columns = new_columns

labels.columns = ["is_faulty", "date_time"]

df = pd.concat([
          secom_data, labels
], axis=1)

C:\Users\Prajakta B\AppData\Local\Temp\ipykernel_10720\4161458270.py:6: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  secom_data = pd.read_csv('C:/Users/Prajakta B/Desktop/SECOM_DATA/secom/secom.data', delim_whitespace=True, header=None)
C:\Users\Prajakta B\AppData\Local\Temp\ipykernel_10720\4161458270.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  labels = pd.read_csv('C:/Users/Prajakta B/Desktop/SECOM_DATA/secom/secom_labels.data', delim_whitespace=True, header=None)


In [108]:
# Test & train dataset split with stratified sampling
from sklearn.model_selection import train_test_split

feature_colms = [col for col in df.columns if col not in ['is_faulty']]
# Features: all columns starting with 'feature_'
X = df[feature_colms]

# Target: 'is_faulty'
y = df['is_faulty']

# Show frequency distribution before sampling
print("Frequency distribution BEFORE sampling:")
print(y.value_counts(normalize=True).rename('proportion').to_frame())
print(y.value_counts().rename('count').to_frame())
print('-' * 40)

# Stratified train-test split (67% train, 33% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20,        # 20% test, 80% train
    random_state=42,       # seed value
    stratify=y             # Stratified split based on is_faulty
)


# Show frequency distribution in train set
print("Frequency distribution in TRAIN set:")
print(y_train.value_counts(normalize=True).rename('proportion').to_frame())
print(y_train.value_counts().rename('count').to_frame())
print('-' * 40)

# Show frequency distribution in test set
print("Frequency distribution in TEST set:")
print(y_test.value_counts(normalize=True).rename('proportion').to_frame())
print(y_test.value_counts().rename('count').to_frame())
print('-' * 40)

train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

train_df_clean = pd.concat([X_train, y_train], axis=1)
# Display datasets
# train_df
# test_df 
# train_df_clean

Frequency distribution BEFORE sampling:
           proportion
is_faulty            
-1           0.933631
 1           0.066369
           count
is_faulty       
-1          1463
 1           104
----------------------------------------
Frequency distribution in TRAIN set:
           proportion
is_faulty            
-1           0.933759
 1           0.066241
           count
is_faulty       
-1          1170
 1            83
----------------------------------------
Frequency distribution in TEST set:
           proportion
is_faulty            
-1           0.933121
 1           0.066879
           count
is_faulty       
-1           293
 1            21
----------------------------------------


In [109]:
# Data cleaning in train dataset train_df_clean
# zero volatality columns

# Count initial columns (including target)
initial_col_count = train_df_clean.shape[1]
print(f"\nInitial number of columns: {initial_col_count}")

# Identify feature columns (exclude target)
feature_cols = [col for col in df.columns if col not in ['is_faulty', 'date_time']]

# Find zero-volatility columns (std = 0)
stds = train_df_clean[feature_cols].std()
zero_vol_cols = stds[stds == 0].index.tolist()

# Remove zero-volatility columns
train_df_clean = train_df_clean.drop(columns=zero_vol_cols)

# Count final columns (including target)
final_col_count = train_df_clean.shape[1]
print(f"Number of zero-volatility columns removed: {len(zero_vol_cols)}")
print(f"Final number of columns: {final_col_count}")
# print(train_df_clean)



Initial number of columns: 592
Number of zero-volatility columns removed: 116
Final number of columns: 476


In [110]:
# missing values column dropping
# Count columns before removal
before_cols = train_df_clean.shape[1]
print(f"Number of columns before removal: {before_cols}")

# Calculate the percentage of missing values in each column
missing_percent = train_df_clean.isnull().mean()


# Identify columns with more than 50% missing values
cols_to_drop = missing_percent[missing_percent > 0.44].index.tolist()

print(f"Number of columns with >50% missing values: {len(cols_to_drop)}")
print("Columns to drop due to missing values >50%:")
print(cols_to_drop)

# Drop these columns
train_df_clean = train_df_clean.drop(columns=cols_to_drop)

# Count columns after removal
after_cols = train_df_clean.shape[1]
print(f"Number of columns after removal: {after_cols}")
print(f"Number of columns removed: {before_cols - after_cols}")


Number of columns before removal: 476
Number of columns with >50% missing values: 32
Columns to drop due to missing values >50%:
['feature_73', 'feature_74', 'feature_86', 'feature_110', 'feature_111', 'feature_112', 'feature_113', 'feature_158', 'feature_159', 'feature_221', 'feature_245', 'feature_246', 'feature_247', 'feature_248', 'feature_293', 'feature_294', 'feature_346', 'feature_347', 'feature_359', 'feature_383', 'feature_384', 'feature_385', 'feature_386', 'feature_493', 'feature_517', 'feature_518', 'feature_519', 'feature_520', 'feature_579', 'feature_580', 'feature_581', 'feature_582']
Number of columns after removal: 444
Number of columns removed: 32


In [111]:
# # Handle missing values with KNN
# from sklearn.impute import KNNImputer

# # Identify feature columns (exclude target)
# feature_cols = [col for col in train_df_clean.columns if col not in ['is_faulty', 'date_time']]

# # Select only numeric columns among features
# numeric_feature_cols = train_df_clean[feature_cols].select_dtypes(include=['float64', 'int64']).columns

# # Store 'is_faulty' and 'date_time' separately
# meta_cols = train_df_clean[['is_faulty', 'date_time']].copy()

# # Apply KNN imputer only to numeric feature columns
# imputer = KNNImputer(n_neighbors=3)
# imputed_features = pd.DataFrame(
#     imputer.fit_transform(train_df_clean[numeric_feature_cols]),
#     columns=numeric_feature_cols,
#     index=train_df_clean.index
# )

# # If you have any non-numeric feature columns, add them back (if needed)
# non_numeric_cols = [col for col in feature_cols if col not in numeric_feature_cols]
# imputed_df = pd.concat([imputed_features, train_df_clean[non_numeric_cols]], axis=1)

# # Append 'is_faulty' and 'date_time' columns back
# train_df_clean = pd.concat([imputed_df, meta_cols], axis=1)

# print(train_df_clean)


In [ ]:
# High Correlation columns

from sklearn.feature_selection import mutual_info_classif
import numpy as np
import pandas as pd

# Initial column count
initial_cols = train_df_clean.shape[1]

# Identify feature columns (exclude target and date)
feature_cols = [col for col in train_df_clean.columns if col not in ['is_faulty', 'date_time']]

# Ensure all features are numeric 
X = train_df_clean[feature_cols]
if X.select_dtypes(include=['object', 'category']).shape[1] > 0:
    X = pd.get_dummies(X)
X = X.fillna(0)

# Compute MI scores
y = train_df_clean['is_faulty'].values
mi_scores = mutual_info_classif(X, y)
mi_series = pd.Series(mi_scores, index=X.columns)

# Compute correlation matrix and its upper triangle
corr_matrix = X.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

threshold = 0.8
to_drop = set()

# For each pair of highly correlated columns, drop the one with lower MI
for col in upper.columns:
    for row in upper.index:
        if pd.notnull(upper.loc[row, col]) and upper.loc[row, col] > threshold:
            if mi_series[col] < mi_series[row]:
                to_drop.add(col)
            else:
                to_drop.add(row)

# Drop the selected columns
train_df_clean = train_df_clean.drop(columns=list(to_drop))

# Print results
print(f"Columns dropped due to high correlation: {len(to_drop)}")
print("\nList of dropped columns:")
print(list(to_drop))

# Final count
final_cols = train_df_clean.shape[1]
print(f"\nInitial columns: {initial_cols}")
print(f"Final columns: {final_cols}")






In [113]:
# 3 sigma 
import numpy as np
from scipy import stats

# Select numeric columns, excluding target and date
numeric_cols = train_df_clean.select_dtypes(include=[np.number]).columns
numeric_cols = [col for col in numeric_cols if col not in ['is_faulty', 'date_time']]

# Compute z-scores for all numeric columns
z_scores = stats.zscore(train_df_clean[numeric_cols], nan_policy='omit')

# Make a copy to cap outliers
# train_df_capped = train_df_clean.copy()

# # Capping function using z-scores
# for idx, col in enumerate(numeric_cols):
#     col_mean = train_df_clean[col].mean()
#     col_std = train_df_clean[col].std()
#     lower = col_mean - 3 * col_std
#     upper = col_mean + 3 * col_std

#     # Find outliers using z-score
#     outliers_high = z_scores[:, idx] > 3
#     outliers_low = z_scores[:, idx] < -3

#     # Cap high outliers
#     train_df_capped.loc[outliers_high, col] = upper
#     # Cap low outliers
#     train_df_capped.loc[outliers_low, col] = lower

# print("Capping done using 3-sigma rule with z-scores.")

# # Optionally, show how many values were capped per column
# capped_counts = {}
# for idx, col in enumerate(numeric_cols):
#     capped_counts[col] = ((z_scores[:, idx] > 3) | (z_scores[:, idx] < -3)).sum()

# print("Number of capped values per column (3-sigma rule):")
# print({k: v for k, v in capped_counts.items() if v > 0})




In [114]:
#4 sigma
# Select numeric columns, excluding target and date if needed
# numeric_cols = train_df_clean.select_dtypes(include=[np.number]).columns
# numeric_cols = [col for col in numeric_cols if col not in ['is_faulty', 'date_time']]

# outlier_columns = []

# for col in numeric_cols:
#     data = train_df_clean[col]
#     mean = data.mean()
#     std = data.std()
#     lower = mean - 4 * std
#     upper = mean + 4 * std
#     outliers = (data < lower) | (data > upper)
#     if outliers.any():
#         outlier_columns.append(col)
        


# print("Columns containing outliers (3-sigma rule):")
# print(outlier_columns)
# print("count:")
# print(len(outlier_columns))




In [115]:
# # Handle missing values with KNN
from sklearn.impute import KNNImputer

# Identify feature columns (exclude target)
feature_cols = [col for col in train_df_clean.columns if col not in ['is_faulty', 'date_time']]

# Select only numeric columns among features
numeric_feature_cols = train_df_clean[feature_cols].select_dtypes(include=['float64', 'int64']).columns

# Store 'is_faulty' and 'date_time' separately
meta_cols = train_df_clean[['is_faulty', 'date_time']].copy()

# Apply KNN imputer only to numeric feature columns
imputer = KNNImputer(n_neighbors=5)
imputed_features = pd.DataFrame(
    imputer.fit_transform(train_df_clean[numeric_feature_cols]),
    columns=numeric_feature_cols,
    index=train_df_clean.index
)

# If you have any non-numeric feature columns, add them back (if needed)
non_numeric_cols = [col for col in feature_cols if col not in numeric_feature_cols]
imputed_df = pd.concat([imputed_features, train_df_clean[non_numeric_cols]], axis=1)

# Append 'is_faulty' and 'date_time' columns back
train_df_clean = pd.concat([imputed_df, meta_cols], axis=1)

print(train_df_clean)

      feature_1  feature_2  feature_3  feature_4  feature_5  feature_7  \
1198    3075.32    2491.07  2185.1000  1201.0491     0.7821   105.8489   
436     3071.58    2489.47  2217.3777  1425.1041     1.7585   106.2556   
635     3017.53    2524.09  2201.0667   880.2317     1.4148   106.5478   
996     2901.62    2569.45  2223.9000  1745.3724     1.9974    96.7567   
782     2982.59    2466.86  2117.5889   894.0996     1.4330   106.4944   
...         ...        ...        ...        ...        ...        ...   
180     3058.89    2504.38  2221.9444  1551.6947     1.5296    99.2678   
365     2988.92    2460.91  2178.0778   941.9524     0.8039   104.0167   
1420    2975.74    2517.35  2162.5556  1041.0369     1.4305   100.4111   
113     2928.16    2523.21  2210.6111  1184.6481     1.2577   102.9356   
470     2929.84    2504.50  2183.3111  1588.5090     1.6269   102.8467   

      feature_8  feature_9  feature_10  feature_11  ...  feature_583  \
1198     0.1208     1.4002     -0.0151 

In [116]:
# Modelling

import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import classification_report, accuracy_score, f1_score


# --- Data Preparation (as in your code) ---
# Identify feature columns (exclude target)
feature_cols = [col for col in train_df_clean.columns if col not in ['is_faulty', 'date_time']]

# Select only numeric columns among features
numeric_feature_cols = train_df_clean[feature_cols].select_dtypes(include=['float64', 'int64']).columns

# Store 'is_faulty' and 'date_time' separately
meta_cols = train_df_clean[['is_faulty', 'date_time']].copy()

# Apply KNN imputer only to numeric feature columns
# imputer = KNNImputer(n_neighbors=5)
# imputed_features = pd.DataFrame(
#     imputer.fit_transform(train_df_clean[numeric_feature_cols]),
#     columns=numeric_feature_cols,
#     index=train_df_clean.index
# )

# If you have any non-numeric feature columns, add them back (if needed)
non_numeric_cols = [col for col in feature_cols if col not in numeric_feature_cols]
imputed_df = pd.concat([imputed_features, train_df_clean[non_numeric_cols]], axis=1)

# Append 'is_faulty' and 'date_time' columns back
train_df_clean = pd.concat([imputed_df, meta_cols], axis=1)

# --- Modeling Setup ---
X = train_df_clean[numeric_feature_cols]
y = train_df_clean['is_faulty']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- Define Models ---
models = {
    'Logistic Regression': LogisticRegression(max_iter=2000, solver='lbfgs'),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'SVM (RBF kernel)': SVC(kernel='rbf', probability=True, random_state=42)
}


# --- Train, Cross-Validate, and Evaluate Each Model ---
for name, model in models.items():
    print(f"\n--- {name} ---")
    # Cross-validation with F1 score (macro average for imbalanced classes)
    cv_f1 = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='f1_macro')
    print(f"Cross-validated F1 (macro) Scores: {cv_f1}")
    print(f"Mean CV F1 (macro): {cv_f1.mean():.4f}")

    # Fit on training data
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    # F1 score on test set
    test_f1 = f1_score(y_test, y_pred, average='macro')
    print("Test Set F1 (macro):", f"{test_f1:.4f}")
    print("Test Set Accuracy:", accuracy_score(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))


--- Logistic Regression ---
Cross-validated F1 (macro) Scores: [0.55432373 0.6097809  0.57757296 0.57281065 0.56175758]
Mean CV F1 (macro): 0.5752
Test Set F1 (macro): 0.5899
Test Set Accuracy: 0.896414342629482
Classification Report:
               precision    recall  f1-score   support

          -1       0.94      0.94      0.94       234
           1       0.24      0.24      0.24        17

    accuracy                           0.90       251
   macro avg       0.59      0.59      0.59       251
weighted avg       0.90      0.90      0.90       251


--- Random Forest ---
Cross-validated F1 (macro) Scores: [0.48329049 0.48195876 0.48186528 0.48320413 0.48320413]
Mean CV F1 (macro): 0.4827
Test Set F1 (macro): 0.4825
Test Set Accuracy: 0.9322709163346613
Classification Report:
               precision    recall  f1-score   support

          -1       0.93      1.00      0.96       234
           1       0.00      0.00      0.00        17

    accuracy                           0

C:\Users\Prajakta B\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Prajakta B\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Prajakta B\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metric

Cross-validated F1 (macro) Scores: [0.48329049 0.48195876 0.48320413 0.48320413 0.48320413]
Mean CV F1 (macro): 0.4830
Test Set F1 (macro): 0.4825
Test Set Accuracy: 0.9322709163346613
Classification Report:
               precision    recall  f1-score   support

          -1       0.93      1.00      0.96       234
           1       0.00      0.00      0.00        17

    accuracy                           0.93       251
   macro avg       0.47      0.50      0.48       251
weighted avg       0.87      0.93      0.90       251



C:\Users\Prajakta B\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Prajakta B\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Prajakta B\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metric

In [117]:
# y_train_final = train_df_clean["is_faulty"]
# X_train_final = train_df_clean.drop(columns=["is_faulty","date_time"])
# # X_test_final = X_test_imputed.drop(columns=high_corr_to_drop)

# # y_train_arr = y_train.drop(columns='date_time')
# y_train_final = y_train_final.to_numpy()

# # Numerical Columns
# numerical_columns = X_train_final.columns.to_list()

# # Numerical Preprocessing Pipeline: Impute missing values with median and scale
# numerical_pipeline = Pipeline(
#     steps=[
#         #('imputer', SimpleImputer(strategy='median')),
#         ('scaler', MinMaxScaler())
#     ]
# )

# # Combine into a Column Transformer
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num_pipeline', numerical_pipeline, numerical_columns)
#     ]
# )

# # %%
# # Define Models
# models = {
#     'Logistic Regression': LogisticRegression(max_iter=1000, class_weight='balanced', n_jobs=-1),
#     'Random Forest': RandomForestClassifier(class_weight='balanced', n_jobs=-1),
#     'SVM': SVC(class_weight='balanced')
# }

# # Define scoring metrics for imbalanced data (focus on class 1 = failure)
# scoring = {
#     'precision': make_scorer(precision_score, average='binary', pos_label=1),
#     'recall': make_scorer(recall_score, average='binary', pos_label=1),
#     'f1': make_scorer(f1_score, average='binary', pos_label=1)
# }

# # Stratified cross-validation
# cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# # Run CV for each model
# results = []

# for model_name, model in models.items():
#     pipeline = Pipeline(steps=[
#         ('preprocessing', preprocessor),
#         ('classifier', model)
#     ])

#     print(f"Training {model_name}...")
#     start_time = time.time()
    
#     y_pred = cross_val_predict(pipeline, X_train_final, y_train_final, cv=cv)
#     execution_time = round(time.time() - start_time, 2)
    
#     # Compute Metrics (still targeting Fail class = 1 as positive class)
#     precision = round(precision_score(y_train_final, y_pred, pos_label=1), 4)
#     recall = round(recall_score(y_train_final, y_pred, pos_label=1), 4)
#     f1 = round(f1_score(y_train_final, y_pred, pos_label=1), 4)

#     # Standard format: [[TN, FP], [FN, TP]]
#     # Negative class: -1 (Pass), Positive class: 1 (Fail)
#     cm_raw = confusion_matrix(y_train_final, y_pred, labels=[-1, 1])
#     tn, fp = cm_raw[0]
#     fn, tp = cm_raw[1]
#     cm = np.array([[tn, fp],
#                    [fn, tp]])


#     results.append({
#         'Model': model_name,
#         'Precision': precision,
#         'Recall': recall,
#         'F1 Score': f1,
#         'Confusion Matrix': cm,
#         'Execution Time (s)': execution_time
#     })

# results_df = pd.DataFrame(results).sort_values(by='F1 Score', ascending=False)
# print("\n📈 Model Comparison:\n")
# print(results_df)
 